Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
def forward_prop(input_dataset, input_weights, input_biases, output_weights, output_biases, keep_prob):
    input_out = tf.matmul(input_dataset, input_weights) + input_biases
    hidden_out = tf.nn.relu(input_out)
    # Add dropout 
    dropout_layer = tf.nn.dropout(hidden_out, keep_prob)
    logits = tf.matmul(dropout_layer, output_weights) + output_biases  # output of hidden layer
    return logits


batch_size = 128
beta = 0.1  # hyper-param of L2 regulization
# learning_rate = 0.1  

num_hidden_nodes = 1024  # number of ReLU units in hidden layer 

graph = tf.Graph()
with graph.as_default():
    global_step = tf.Variable(0)  # count how many steps has been made
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Network structure
    # Input layer
    input_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    input_biases = tf.Variable(tf.zeros([num_hidden_nodes]))
    # output layer
    output_weights = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    
    # keep probability
    keep_prob = tf.placeholder(tf.float32)
    
    # Computing graph
    logits = forward_prop(tf_train_dataset, input_weights, input_biases, output_weights, output_biases, keep_prob)
    
    # Loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
            + beta * 0.5 * (tf.nn.l2_loss(input_weights) + tf.nn.l2_loss(input_biases) 
                      + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)) 
    # Optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 50, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Prediction for training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, 
                                                  input_weights, input_biases, output_weights, output_biases, 1))
    test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, 
                                                  input_weights, input_biases, output_weights, output_biases, 1))
    

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    # Initialized tf.placeholder
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    #Check out the test accuracy
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16189.170898
Minibatch accuracy: 10.9%
Validation accuracy: 30.2%
Minibatch loss at step 500: 1.198824
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 1.205080
Minibatch accuracy: 78.1%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 1.103737
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 1.072599
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 1.226796
Minibatch accuracy: 75.8%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 1.069513
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Test accuracy: 88.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [81]:
batch_size = 512  # for Stochastic Gradient Descent
num_relu_1 = 170
num_relu_2 = 200
# num_relu_2 = num_labels

# Regulization param
beta = 0.01
# learning_rate = 0.1

graph_deep = tf.Graph()
with graph_deep.as_default():   
    def forward_prop(input_data, weights, biases, keep_prob):
        out_1 = tf.matmul(input_data, weights[0]) + biases[0]
        out_2 = tf.nn.dropout(tf.nn.relu(out_1), keep_prob)  # dropout here
        out_3 = tf.matmul(out_2, weights[1]) + biases[1]
        out_4 = tf.nn.dropout(tf.nn.relu(out_3), keep_prob)  # dropout here
        out_5 = tf.matmul(out_4, weights[2]) + biases[2]  # logits
        return out_5

    # Input data
    gr_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    gr_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    gr_valid_dataset = tf.constant(valid_dataset)
    gr_test_dataset = tf.constant(test_dataset)
    
    # Network structure
    # Layer 1 - linear
    layer_1_linear_wei = tf.Variable(tf.truncated_normal([image_size * image_size, num_relu_1]))
    layer_1_linear_bia = tf.Variable(tf.zeros([num_relu_1]))
    # Layer 2 - ReLU
    # Layer 3 - linear
    layer_3_linear_wei = tf.Variable(tf.truncated_normal([num_relu_1, num_relu_2]))
    layer_3_linear_bia = tf.Variable(tf.zeros([num_relu_2]))
    # Layer 4 - ReLU
    # Layer 5 - linear
    layer_5_linear_wei = tf.Variable(tf.truncated_normal([num_relu_2, num_labels]))
    layer_5_linear_bia = tf.Variable(tf.zeros([num_labels]))
    # Store weights & biases into lists
    weights = [layer_1_linear_wei, layer_3_linear_wei, layer_5_linear_wei]
    biases = [layer_1_linear_bia, layer_3_linear_bia, layer_5_linear_bia]
    
    # Compute loss function
    logits_deep = forward_prop(gr_train_dataset, weights, biases, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=gr_train_labels, logits=logits_deep))
    # Add L2 regularization to loss function
    for wei_bia in zip(weights, biases):
        loss += beta * (tf.nn.l2_loss(wei_bia[0]) + tf.nn.l2_loss(wei_bia[1]))
        
    # Optimizer
    # compute learning rate
    global_step = tf.Variable(0)  # count how many steps has been made
    learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 1.0)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)
    
    # compute network prediction
    train_prediction = tf.nn.softmax(logits_deep)
    valid_prediction = tf.nn.softmax(forward_prop(gr_valid_dataset, weights, biases, 1))
    test_prediction = tf.nn.softmax(forward_prop(gr_test_dataset, weights, biases, 1))
    

In [82]:
# TRAINING
num_steps = 3001

with tf.Session(graph=graph_deep) as session:
    # Initialized tf.placeholder
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {gr_train_dataset : batch_data, gr_train_labels : batch_labels}
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Step:%d\t\t Loss:%.3f\t\t Train acc:%.1f\t\t Valid acc:%.1f" % 
                  (step, l, accuracy(predictions, batch_labels), accuracy(valid_prediction.eval(), valid_labels)))
#             print("Minibatch loss at step %d: %f" % (step, l))
#             print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#             print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    #Check out the test accuracy
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Step:0		 Loss:3338.534		 Train acc:10.0		 Valid acc:25.8
Step:500		 Loss:236.603		 Train acc:25.6		 Valid acc:23.7
Step:1000		 Loss:87.573		 Train acc:49.6		 Valid acc:66.7
Step:1500		 Loss:32.649		 Train acc:70.9		 Valid acc:77.9
Step:2000		 Loss:12.514		 Train acc:77.7		 Valid acc:81.7
Step:2500		 Loss:5.170		 Train acc:77.3		 Valid acc:82.9
Step:3000		 Loss:2.412		 Train acc:80.9		 Valid acc:83.7
Test accuracy: 89.5%
